In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import random
from tqdm import tqdm
import torch.nn as nn
import torch.sparse as sp
import seaborn as sns
sns.set_theme()

In [2]:
def sparse_drop(feature, drop_out):
    tem = torch.rand((feature._nnz())) # nnz is number of non zero elements
    feature._values()[tem < drop_out] = 0
    return feature
    
class GCMC(nn.Module):
    def __init__(self, feature_u, 
                 feature_v, 
                 feature_dim, 
                 hidden_dim, 
                 rate_num, 
                 all_M_u, 
                 all_M_v, 
                 side_hidden_dim, 
                 side_feature_u, 
                 side_feature_v, 
                 use_side : bool,
                 out_dim, 
                 drop_out = 0.0):
        super(GCMC, self).__init__()
        
        self.drop_out = drop_out
        
        side_feature_u_dim = side_feature_u.shape[1]
        side_feature_v_dim = side_feature_v.shape[1]
        self.use_side = use_side

        self.feature_u = feature_u
        self.feature_v = feature_v
        self.rate_num = rate_num
        
        self.num_user = feature_u.shape[0]
        self.num_item = feature_v.shape[1]
        
        self.side_feature_u = side_feature_u
        self.side_feature_v = side_feature_v
        
        self.W = nn.Parameter(torch.randn(rate_num, feature_dim, hidden_dim))
        nn.init.kaiming_normal_(self.W, mode = 'fan_out', nonlinearity = 'relu')
        
        self.all_M_u = all_M_u
        self.all_M_v = all_M_v
        
        self.reLU = nn.ReLU()
        
        if use_side:
            self.linear_layer_side_u = nn.Sequential(*[nn.Linear(side_feature_u_dim, side_hidden_dim, bias = True), 
                                                       nn.BatchNorm1d(side_hidden_dim), nn.ReLU()])
            self.linear_layer_side_v = nn.Sequential(*[nn.Linear(side_feature_v_dim, side_hidden_dim, bias = True), 
                                                       nn.BatchNorm1d(side_hidden_dim), nn.ReLU()])
    
            self.linear_cat_u = nn.Sequential(*[nn.Linear(rate_num * hidden_dim * 2 + side_hidden_dim, out_dim, bias = True), 
                                                nn.BatchNorm1d(out_dim), nn.ReLU()])
            self.linear_cat_v = nn.Sequential(*[nn.Linear(rate_num * hidden_dim * 2 + side_hidden_dim, out_dim, bias = True), 
                                                nn.BatchNorm1d(out_dim), nn.ReLU()])    
        else:
            
            self.linear_cat_u = nn.Sequential(*[nn.Linear(rate_num * hidden_dim * 2, out_dim, bias = True), 
                                                nn.BatchNorm1d(out_dim), nn.ReLU()])
            self.linear_cat_v = nn.Sequential(*[nn.Linear(rate_num * hidden_dim * 2, out_dim, bias = True), 
                                                nn.BatchNorm1d(out_dim), nn.ReLU()])
            
        self.Q = nn.Parameter(torch.randn(rate_num, out_dim, out_dim))
        nn.init.orthogonal_(self.Q)
        
        
    def forward(self):

        # Here is the drop + normalization to have no problems with mean
        feature_u_drop = sparse_drop(self.feature_u, self.drop_out) / (1.0 - self.drop_out)
        feature_v_drop = sparse_drop(self.feature_v, self.drop_out) / (1.0 - self.drop_out)
        
        hidden_feature_u = []
        hidden_feature_v = []
        
        W_list = torch.split(self.W, self.rate_num) # we have different W for every rating, as stated in the paper
        W_flat = []
        for i in range(self.rate_num): # iterate over every rating
            Wr = W_list[0][i]
            M_u = self.all_M_u[i]
            M_v = self.all_M_v[i]
            
            # H_u from paper. The embeddings
            hidden_u = sp.mm(feature_v_drop, Wr)
            hidden_u = self.reLU(sp.mm(M_u, hidden_u))
            
            # need to further process M, normalization
            hidden_v = sp.mm(feature_u_drop, Wr)
            hidden_v = self.reLU(sp.mm(M_v, hidden_v))

            
            hidden_feature_u.append(hidden_u)
            hidden_feature_v.append(hidden_v)
            
            W_flat.append(Wr)
            
        hidden_feature_u = torch.cat(hidden_feature_u, dim = 1)
        hidden_feature_v = torch.cat(hidden_feature_v, dim = 1)
        W_flat = torch.cat(W_flat, dim = 1)


        cat_u = torch.cat((hidden_feature_u, torch.mm(self.feature_u, W_flat)), dim = 1)
        cat_v = torch.cat((hidden_feature_v, torch.mm(self.feature_v, W_flat)), dim = 1)
        
        if self.use_side:
            side_hidden_feature_u = self.linear_layer_side_u(self.side_feature_u)
            side_hidden_feature_v = self.linear_layer_side_v(self.side_feature_v)    

            cat_u = torch.cat((cat_u, side_hidden_feature_u), dim = 1)
            cat_v = torch.cat((cat_v, side_hidden_feature_v), dim = 1)
        
        
        embed_u = self.linear_cat_u(cat_u)
        embed_v = self.linear_cat_v(cat_v)
        
        score = []
        Q_list = torch.split(self.Q, self.rate_num)
        for i in range(self.rate_num):
            Qr = Q_list[0][i]
            
            tem = torch.mm(torch.mm(embed_u, Qr), torch.t(embed_v))
            
            score.append(tem)

        score = torch.stack(score)
        return score

In [3]:
ratings_test = torch.load("ratings_test.pt")
test_mask = torch.load("test_mask.pt")

In [4]:
def scores_to_matrix(score):
    sm = nn.Softmax(dim = 0)
    score = sm(score)
    score_list = torch.split(score, 5)
    pred = 0
    for i in range(5):
        pred += (i + 1) * score_list[0][i]
    return pred

In [5]:
net = torch.load("./../models/best_model.pt")
generated_ratings = scores_to_matrix(net.forward())
generated_ratings[~test_mask] = 0 # zero all movies we have seen during the training

num_users, num_items = generated_ratings.shape

In [6]:
generated_ratings[~test_mask] = 0 # zero all movies we have seen during the training

In [7]:
num_users, num_items = generated_ratings.shape

In [14]:
k = 20
_, top_k_indices = torch.topk(generated_ratings, k)
relevant_ratings = ratings_test >= 4

In [15]:
MAP_sum = 0
for user in range(num_users): # for every user
    ap_sum = 0
    counter = 1
    for i, idx in enumerate(top_k_indices[user]): # iterate over suggestions
        if relevant_ratings[user][idx]:
            ap_sum += counter / (i+1)
            counter += 1
    if relevant_ratings[user].sum() == 0:
        ap_sum = 1
    else:
        ap_sum/=min(k, relevant_ratings[user].sum())
    MAP_sum += ap_sum
print(MAP_sum / num_users)
        

tensor(0.7821)
